In [ ]:
import pandas as pd

In [ ]:
%%capture
!pip install gdown

In [ ]:
!gdown "https://drive.google.com/uc?id=1i-sTZzhYwf_ShhTCFNGyMNugaVIkMix6"


Downloading...
From: https://drive.google.com/uc?id=1i-sTZzhYwf_ShhTCFNGyMNugaVIkMix6
To: /content/Eval.zip
100% 1.29M/1.29M [00:00<00:00, 68.2MB/s]


In [ ]:
! unzip /content/Eval.zip

Archive:  /content/Eval.zip
   creating: Eval/
   creating: Eval/model_base_7/
  inflating: Eval/model_base_7/base_model_outputs.csv  
  inflating: Eval/model_base_7/metrics.json  
  inflating: Eval/model_base_7/desktop.ini  
   creating: Eval/model_base_1/
  inflating: Eval/model_base_1/metric base model 1.json  
  inflating: Eval/model_base_1/base_model_outputs.csv  
  inflating: Eval/model_base_1/desktop.ini  
   creating: Eval/model_ft_1/
  inflating: Eval/model_ft_1/fine tune base model.json  
  inflating: Eval/model_ft_1/fine_tuned_outputs.csv  
  inflating: Eval/model_ft_1/desktop.ini  
   creating: Eval/model_ft_7/
  inflating: Eval/model_ft_7/fine_tuned_outputs.csv  
  inflating: Eval/model_ft_7/metrics.json  
  inflating: Eval/model_ft_7/desktop.ini  
   creating: Eval/model_base_8/
  inflating: Eval/model_base_8/base_model_outputs.csv  
  inflating: Eval/model_base_8/metrics.json  
  inflating: Eval/model_base_8/desktop.ini  
   creating: Eval/model_base_5/
   creating: Eval

In [ ]:
# prompt: import bert score, bleu, rougr
%%capture
!pip install bert-score
!pip install evaluate
!pip install rouge_score

In [ ]:
import bert_score
from evaluate import load
import pandas as pd

In [ ]:
import evaluate

In [ ]:
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import evaluate

bleu = evaluate.load("bleu")

predictions = ["the cat is on the mat"]
references = [["there is a cat on the mat"]]

# BLEU-1
result_bleu1 = bleu.compute(predictions=predictions, references=references, max_order=1)
print(f"BLEU-1: {result_bleu1['bleu']:.4f}")

# BLEU-2
result_bleu2 = bleu.compute(predictions=predictions, references=references, max_order=2)
print(f"BLEU-2: {result_bleu2['bleu']:.4f}")

# BLEU-3
result_bleu3 = bleu.compute(predictions=predictions, references=references, max_order=3)
print(f"BLEU-3: {result_bleu3['bleu']:.4f}")

# BLEU-4 (default)
result_bleu4 = bleu.compute(predictions=predictions, references=references)
print(f"BLEU-4: {result_bleu4['bleu']:.4f}")


BLEU-1: 0.7054
BLEU-2: 0.4887
BLEU-3: 0.3697
BLEU-4: 0.0000


In [ ]:
pd.options.display.max_colwidth = 500

Function Compute Bleu score


In [ ]:
import evaluate
import pandas as pd

# Load evaluator sekali saja
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

def compute_bleu_per_row(predictions, references):
    """
    Menghitung BLEU 1-4 untuk setiap pasangan prediksi dan referensi.
    Return: list of dict, tiap dict {'bleu1':..., 'bleu2':..., 'bleu3':..., 'bleu4':...}
    """
    all_scores = []
    for pred, ref in zip(predictions, references):
        scores = {}
        # ref harus list of references, meskipun 1 referensi
        ref_list = [ref] if isinstance(ref, str) else ref
        for n in range(1, 5):
            score = bleu.compute(predictions=[pred], references=[ref_list], max_order=n, smooth=True)
            scores[f'bleu{n}'] = score['bleu']
        all_scores.append(scores)
    return all_scores

Function Compoute Rouge Score

In [ ]:
def compute_rouge_per_row(predictions, references):
    rouge = evaluate.load("rouge")
    scores = []
    for pred, ref in zip(predictions, references):
        score = rouge.compute(predictions=[pred], references=[ref])
        # Ambil langsung fmeasure-nya (hasil evaluate adalah dict float)
        scores.append({
            "rouge1": score["rouge1"],
            "rouge2": score["rouge2"],
            "rougeL": score["rougeL"],
            "rougeLSum": score["rougeLsum"]
        })
    return scores


Bert score multiligual bert

In [ ]:
import evaluate

def compute_bertscore_per_row(predictions, references):
    """
    Menghitung BERTScore per kalimat untuk Bahasa Indonesia menggunakan library evaluate.

    Parameters:
    - predictions: List[str], list kalimat yang dihasilkan model.
    - references: List[List[str]], list referensi yang diharapkan (nested list, satu referensi per kalimat).

    Returns:
    - List of Dicts berisi precision, recall, dan F1 untuk setiap kalimat.
    """
    # Muat evaluator
    bertscore = evaluate.load("bertscore")

    # Pastikan referensi adalah list of strings (flatten)
    flat_references = [ref[0] for ref in references]

    # Hitung skor
    results = bertscore.compute(
        predictions=predictions,
        references=flat_references,
        lang="id"  # Bahasa Indonesia
    )

    # Gabungkan hasil per kalimat
    scores_per_row = []
    for i in range(len(predictions)):
        scores_per_row.append({
            "prediction": predictions[i],
            "reference": flat_references[i],
            "bert_precision": results["precision"][i],
            "bert_recall": results["recall"][i],
            "bert_f1": results["f1"][i],
        })

    return scores_per_row

# Contoh penggunaan
predictions = ["kucing itu duduk di atas tikar"]
references = [["seekor kucing duduk di tikar"]]

scores = compute_bertscore_per_row(predictions, references)

print("BERTScore per row:")
for i, score in enumerate(scores):
    print(f"Row {i+1}:")
    print(f"  Prediction: {score['prediction']}")
    print(f"  Reference : {score['reference']}")
    print(f"  Precision : {score['bert_precision']:.4f}")
    print(f"  Recall    : {score['bert_recall']:.4f}")
    print(f"  F1        : {score['bert_f1']:.4f}")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_REMOVED_BY_SCRIPT' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_REMOVED_BY_SCRIPT]` or `pip install hf_REMOVED_BY_SCRIPT`


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

BERTScore per row:
Row 1:
  Prediction: kucing itu duduk di atas tikar
  Reference : seekor kucing duduk di tikar
  Precision : 0.7809
  Recall    : 0.7824
  F1        : 0.7817


# Model 6

## Base Model 6

In [ ]:
base_6 = pd.read_csv('/content/Eval/model_base_6/content/eval_basemodel_20250525_075214/base_model_outputs.csv')
base_6.sample(1)

,instruction,input,reference,generated,bert_precision,bert_recall,bert_f1
75,"Penyebab perut tidak nyaman di perut, sering berbunyi, mudah lapar dan lemas berlebihan","Penyebab perut tidak nyaman di perut, sering berbunyi, mudah lapar dan lemas berlebihan. dokter yang budiman Saya ingin bertanya mengenai pencernaan. Kurang lebih sudah satu bulan, perut saya ada gangguan. Suka lemas, tiba-tiba sangat lapar, kadang sampai susah beraktifitas. Contoh; Bangun siang pas dzuhur sudah susah bangun, karena saking lemasnya. Padahal jam sembilan atau 10 baru makan. Kalau bulan-bulan kemarin masih normal. Sehari juga 2 kali makan kuat. Kalau sekarang, sehari kayanya h...","rasa tidak nyaman di perut, sering berbunyi, mudah lapar, dan lemas berlebihan yang muncul dalam waktu lama bisa merupakan kondisi yang ringan ataupun berbahaya. beberapa kemungkinan penyebabnya adalah: melalukan diet rendah karbohi , misalnya jika anda terlalu banyak mengkonsumsi buah atau produk susu tertentu malabsorpsi, misalnya jika anda terlalu banyak mengkonsumsi makanan dengan pemanis buatan demam tifoid ( tifus ) gastroenteritis (infeksi saluran cerna) tukak lambung (luka bergaung d...","Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>\n\n### Keluhan:\nPenyebab perut tidak nyaman di perut, sering berbunyi, mudah lapar dan lemas berlebihan\n\n### Detail Pertanyaan:\nPenyebab perut tidak nyaman di perut, sering berbunyi, mudah lapar dan lemas berlebihan. dokter yang budiman Saya ingi...",0.690425,0.683312,0.68685


Bersihin generated

In [ ]:
def get_text_after_jawaban(text):
  """
  Mengambil teks setelah string '### Jawaban:' dalam sebuah string.

  Args:
    text: String input (diasumsikan string, tangani NaN jika perlu).

  Returns:
    String teks setelah '### Jawaban:', atau None jika '### Jawaban:' tidak ditemukan atau input bukan string.
  """
  if not isinstance(text, str): # Menangani jika ada nilai non-string (misalnya NaN)
    return None

  delimiter = "### Jawaban:"
  parts = text.split(delimiter)

  if len(parts) > 1:
    return parts[1].strip()
  else:
    return None

In [ ]:
base_6['generated'] = base_6['generated'].apply(get_text_after_jawaban)

In [ ]:
base_6.head(1)

,instruction,input,reference,generated,bert_precision,bert_recall,bert_f1
0,Solusi atasi sakit maag yang tak kunjung sembuh,"Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosanya sama yaitu saya menderita maag. Oleh dokter yang menangani, saya hanya diberi antasida, omeprazole, dan ranitidin. Saya sudah habiskan obatnya, pola makan pun saya jaga dengan tidak mengkonsumsi makanan pedas, asam, dan kafein namun sampai saat ini tidak ada perubahan. Selama 2 minggu saya sakit berat badan saya turun 1...","usus halus ataupun kerongkongan yang dapat disebabkan kondisi berikut : faktor psikis, yakni kondisi stres emosional akibat pengunaan obat antinyeri non steroid (nsaid) akibat perlukaan pada lambung (tukak lambung/peptik) infeksi bakteri helicobacter pylori (gastritis) dsb sakit maag yang kronis atau berlangsung berat dapat menyebabkan perlukaan pada lambung dan sebaliknya. pada perlukaan yang berat dapat terjadi perforasi (pecahnya) jaringan lambung sehingga menimbulkan gejala batuk/muntah ...","Maag memang bisa menimbulkan gejala yang sangat tidak nyaman, seperti yang Anda alami. Namun, penting untuk diingat bahwa setiap orang berbeda, dan gejala maag bisa bervariasi.\n\nMeskipun Anda telah menjalani pengobatan medis, namun gejala Anda masih berlanjut dan bahkan semakin memburuk. Hal ini mengindikasikan kemungkinan adanya faktor lain yang mempengaruhi kondisi Anda, atau mungkin pengobatan yang sebelumnya tidak efektif.\n\nEndoskopi dan tes darah dapat membantu dokter menentukan pen...",0.678952,0.687113,0.683008


In [ ]:
# Buang baris yang memiliki nan pada kolom generated atau reference
df_clean = base_6.dropna(subset=['generated', 'reference']).copy()

preds = df_clean['generated'].tolist()
refs = df_clean['reference'].tolist()

# Pastikan references berbentuk list of list untuk BLEU
refs_for_bleu = [[ref] for ref in refs]

bleu_scores = compute_bleu_per_row(preds, refs_for_bleu)
rouge_scores = compute_rouge_per_row(preds, refs)

# Buat dict untuk dataframe baru
data_new = {}

for n in range(1, 5):
    data_new[f'bleu{n}'] = [score[f'bleu{n}'] for score in bleu_scores]

data_new['rouge1'] = [score['rouge1'] for score in rouge_scores]
data_new['rouge2'] = [score['rouge2'] for score in rouge_scores]
data_new['rougeL'] = [score['rougeL'] for score in rouge_scores]
data_new['rougeLSum'] = [score['rougeLSum'] for score in rouge_scores]

In [ ]:
bertscore_eval_scores = compute_bertscore_per_row(preds, refs_for_bleu)

In [ ]:
df_bert_score = pd.DataFrame(bertscore_eval_scores)
df_bert_score

,prediction,reference,bert_precision,bert_recall,bert_f1
0,"Maag memang bisa menimbulkan gejala yang sangat tidak nyaman, seperti yang Anda alami. Namun, penting untuk diingat bahwa setiap orang berbeda, dan gejala maag bisa bervariasi.\n\nMeskipun Anda telah menjalani pengobatan medis, namun gejala Anda masih berlanjut dan bahkan semakin memburuk. Hal ini mengindikasikan kemungkinan adanya faktor lain yang mempengaruhi kondisi Anda, atau mungkin pengobatan yang sebelumnya tidak efektif.\n\nEndoskopi dan tes darah dapat membantu dokter menentukan pen...","usus halus ataupun kerongkongan yang dapat disebabkan kondisi berikut : faktor psikis, yakni kondisi stres emosional akibat pengunaan obat antinyeri non steroid (nsaid) akibat perlukaan pada lambung (tukak lambung/peptik) infeksi bakteri helicobacter pylori (gastritis) dsb sakit maag yang kronis atau berlangsung berat dapat menyebabkan perlukaan pada lambung dan sebaliknya. pada perlukaan yang berat dapat terjadi perforasi (pecahnya) jaringan lambung sehingga menimbulkan gejala batuk/muntah ...",0.670203,0.642192,0.655899
1,"Konstipasi adalah kondisi ketika buang air besar menjadi sulit atau tidak teratur. Ada beberapa solusi yang bisa kamu coba untuk mengatasi konstipasi, seperti:\n\n* **Meningkatkan asupan serat**: Makan makanan yang kaya serat seperti buah, sayur, dan biji-bijian dapat membantu melancarkan buang air besar.\n* **Minum cukup air**: Dehidrasi dapat memperburuk konstipasi, jadi pastikan kamu minum cukup air putih setiap hari.\n* **Rutin berolahraga**: Olahraga teratur dapat membantu melancarkan p...",konstipasi adalah keadaan tidak dapat buang air besar secara teratur atau tidak dapat buang air besar sama sekali. konstipasi dapat disebabkan oleh: kurang mimun kurang konsumsi serat kurang aktif bergerak dan berolahraga kelebihan atau kekurangan berat badan kebiasaan menunda buang air besar cemas atau depresi penyumbatan usus gangguan saraf usus efek samping obat-obatan tertentu dsb. sebaiknya anda berkonsultasi dengan dokter di mana dokter akan menganalisis gejala dan melakukan pemeriksaa...,0.672901,0.683458,0.678138
2,"Berdasarkan gejala yang Anda alami, seperti penurunan berat badan, kehilangan nafsu makan, sendawa, nyeri ulu hati dan perut kanan, muntah kekuningan disertai bercak merah seperti darah segar, perlu dilakukan pemeriksaan lebih lanjut untuk memastikan penyebab pasti keluhan Anda. Endoskopi dan tes darah bisa menjadi pilihan untuk membantu dokter menentukan diagnosis dan penanganan yang tepat.\n\nEndoskopi dapat membantu dokter memeriksa kondisi lambung dan usus halus Anda secara langsung untu...","usus halus ataupun kerongkongan yang dapat disebabkan kondisi berikut : faktor psikis, yakni kondisi stres emosional akibat pengunaan obat antinyeri non steroid (nsaid) akibat perlukaan pada lambung (tukak lambung/peptik) infeksi bakteri helicobacter pylori (gastritis) dsb sakit maag yang kronis atau berlangsung berat dapat menyebabkan perlukaan pada lambung dan sebaliknya. pada perlukaan yang berat dapat terjadi perforasi (pecahnya) jaringan lambung sehingga menimbulkan gejala batuk/muntah ...",0.694377,0.638089,0.665044
3,"Konstipasi dapat disebabkan oleh berbagai faktor, termasuk pola makan, gaya hidup, dan kondisi medis tertentu. Jika Anda mengalami kesulitan buang air besar dengan feses yang tidak keras dan cair, namun usus tidak mau mengeluarkannya, beberapa solusi yang dapat Anda coba adalah:\n\n1. **Meningkatkan asupan serat:** Serat dapat membantu memperlancar pencernaan dan mempermudah buang air besar. Konsumsi makanan yang kaya serat seperti buah-buahan, sayuran, biji-bijian, dan kacang-kacangan.\n2. ...",konstipasi adalah keadaan tidak dapat buang air besar secara teratur atau tidak dapat buang air besar sama sekali. konstipasi dapat disebabkan oleh: kurang mimun kurang konsumsi serat kurang aktif bergerak dan berolahraga kelebihan atau kekurangan berat badan kebiasaan menunda buang air besar cemas atau depresi penyumbatan usus gangguan sar

In [ ]:
df_scores = pd.DataFrame(data_new)
df_scores = df_scores * 100
df_scores = pd.concat([df_bert_score, df_scores], axis=1)
df_scores.head(2)

,prediction,reference,bert_precision,bert_recall,bert_f1,bleu1,bleu2,bleu3,bleu4,rouge1,rouge2,rougeL,rougeLSum
0,"Maag memang bisa menimbulkan gejala yang sangat tidak nyaman, seperti yang Anda alami. Namun, penting untuk diingat bahwa setiap orang berbeda, dan gejala maag bisa bervariasi.\n\nMeskipun Anda telah menjalani pengobatan medis, namun gejala Anda masih berlanjut dan bahkan semakin memburuk. Hal ini mengindikasikan kemungkinan adanya faktor lain yang mempengaruhi kondisi Anda, atau mungkin pengobatan yang sebelumnya tidak efektif.\n\nEndoskopi dan tes darah dapat membantu dokter menentukan pen...","usus halus ataupun kerongkongan yang dapat disebabkan kondisi berikut : faktor psikis, yakni kondisi stres emosional akibat pengunaan obat antinyeri non steroid (nsaid) akibat perlukaan pada lambung (tukak lambung/peptik) infeksi bakteri helicobacter pylori (gastritis) dsb sakit maag yang kronis atau berlangsung berat dapat menyebabkan perlukaan pada lambung dan sebaliknya. pada perlukaan yang berat dapat terjadi perforasi (pecahnya) jaringan lambung sehingga menimbulkan gejala batuk/muntah ...",0.670203,0.642192,0.655899,26.358757,8.154402,3.983544,2.118187,35.746606,6.818182,14.479638,19.457014
1,"Konstipasi adalah kondisi ketika buang air besar menjadi sulit atau tidak teratur. Ada beberapa solusi yang bisa kamu coba untuk mengatasi konstipasi, seperti:\n\n* **Meningkatkan asupan serat**: Makan makanan yang kaya serat seperti buah, sayur, dan biji-bijian dapat membantu melancarkan buang air besar.\n* **Minum cukup air**: Dehidrasi dapat memperburuk konstipasi, jadi pastikan kamu minum cukup air putih setiap hari.\n* **Rutin berolahraga**: Olahraga teratur dapat membantu melancarkan p...",konstipasi adalah keadaan tidak dapat buang air besar secara teratur atau tidak dapat buang air besar sama sekali. konstipasi dapat disebabkan oleh: kurang mimun kurang konsumsi serat kurang aktif bergerak dan berolahraga kelebihan atau kekurangan berat badan kebiasaan menunda buang air besar cemas atau depresi penyumbatan usus gangguan saraf usus efek samping obat-obatan tertentu dsb. sebaiknya anda berkonsultasi dengan dokter di mana dokter akan menganalisis gejala dan melakukan pemeriksaa...,0.672901,0.683458,0.678138,32.972973,13.386596,7.316719,3.830182,37.370242,6.968641,17.993080,26.297578


In [ ]:
# Calculate the mean of the numerical columns only
numerical_cols = ['bert_precision', 'bert_recall', 'bert_f1', 'bleu1', 'bleu2', 'bleu3', 'bleu4', 'rouge1', 'rouge2', 'rougeL', 'rougeLSum']
mean_scores = df_scores[numerical_cols].mean()

print("\nRata-rata Skor:")
mean_scores


Rata-rata Skor:


,0
bert_precision,0.681518
bert_recall,0.669774
bert_f1,0.675253
bleu1,22.639632
bleu2,9.460875
bleu3,4.627718
bleu4,2.704909
rouge1,29.472935
rouge2,5.410137
rougeL,14.844500


In [ ]:
import os
os.makedirs('Hasil Eval revisi')
mean_scores.to_csv('Hasil Eval revisi/eval_basemodel_6_mean.csv')

In [ ]:
df_scores = pd.concat([base_6[['input']], df_scores], axis=1)
df_scores.head(2)

,input,prediction,reference,bert_precision,bert_recall,bert_f1,bleu1,bleu2,bleu3,bleu4,rouge1,rouge2,rougeL,rougeLSum
0,"Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosanya sama yaitu saya menderita maag. Oleh dokter yang menangani, saya hanya diberi antasida, omeprazole, dan ranitidin. Saya sudah habiskan obatnya, pola makan pun saya jaga dengan tidak mengkonsumsi makanan pedas, asam, dan kafein namun sampai saat ini tidak ada perubahan. Selama 2 minggu saya sakit berat badan saya turun 1...","Maag memang bisa menimbulkan gejala yang sangat tidak nyaman, seperti yang Anda alami. Namun, penting untuk diingat bahwa setiap orang berbeda, dan gejala maag bisa bervariasi.\n\nMeskipun Anda telah menjalani pengobatan medis, namun gejala Anda masih berlanjut dan bahkan semakin memburuk. Hal ini mengindikasikan kemungkinan adanya faktor lain yang mempengaruhi kondisi Anda, atau mungkin pengobatan yang sebelumnya tidak efektif.\n\nEndoskopi dan tes darah dapat membantu dokter menentukan pen...","usus halus ataupun kerongkongan yang dapat disebabkan kondisi berikut : faktor psikis, yakni kondisi stres emosional akibat pengunaan obat antinyeri non steroid (nsaid) akibat perlukaan pada lambung (tukak lambung/peptik) infeksi bakteri helicobacter pylori (gastritis) dsb sakit maag yang kronis atau berlangsung berat dapat menyebabkan perlukaan pada lambung dan sebaliknya. pada perlukaan yang berat dapat terjadi perforasi (pecahnya) jaringan lambung sehingga menimbulkan gejala batuk/muntah ...",0.670203,0.642192,0.655899,26.358757,8.154402,3.983544,2.118187,35.746606,6.818182,14.479638,19.457014
1,"solusi untuk mengatasi konstipasi. dok kalo bab feses saya enggak keras, cair tapi usus saya enggak mau untuk ngeluarinnya. kenapa ya dok?","Konstipasi adalah kondisi ketika buang air besar menjadi sulit atau tidak teratur. Ada beberapa solusi yang bisa kamu coba untuk mengatasi konstipasi, seperti:\n\n* **Meningkatkan asupan serat**: Makan makanan yang kaya serat seperti buah, sayur, dan biji-bijian dapat membantu melancarkan buang air besar.\n* **Minum cukup air**: Dehidrasi dapat memperburuk konstipasi, jadi pastikan kamu minum cukup air putih setiap hari.\n* **Rutin berolahraga**: Olahraga teratur dapat membantu melancarkan p...",konstipasi adalah keadaan tidak dapat buang air besar secara teratur atau tidak dapat buang air besar sama sekali. konstipasi dapat disebabkan oleh: kurang mimun kurang konsumsi serat kurang aktif bergerak dan berolahraga kelebihan atau kekurangan berat badan kebiasaan menunda buang air besar cemas atau depresi penyumbatan usus gangguan saraf usus efek samping obat-obatan tertentu dsb. sebaiknya anda berkonsultasi dengan dokter di mana dokter akan menganalisis gejala dan melakukan pemeriksaa...,0.672901,0.683458,0.678138,32.972973,13.386596,7.316719,3.830182,37.370242,6.968641,17.993080,26.297578


In [ ]:
df_scores.to_csv('Hasil Eval revisi/eval_basemodel_6.csv')

## Fine Tune Model 6

In [ ]:
def evaluate_generation_scores(data):
    # Buang baris yang memiliki NaN pada kolom generated atau reference
    df_clean = data.dropna(subset=['generated', 'reference']).copy()

    preds = df_clean['generated'].tolist()
    refs = df_clean['reference'].tolist()

    # Pastikan references berbentuk list of list untuk BLEU
    refs_for_bleu = [[ref] for ref in refs]

    # Hitung BLEU dan ROUGE per baris
    bleu_scores = compute_bleu_per_row(preds, refs_for_bleu)
    rouge_scores = compute_rouge_per_row(preds, refs)

    # Hitung BERTScore per baris
    bert_scores = compute_bertscore_per_row(preds, refs_for_bleu)

    # Siapkan dictionary untuk skor
    data_new = {}

    for n in range(1, 5):
        data_new[f'bleu{n}'] = [score[f'bleu{n}'] for score in bleu_scores]

    data_new['rouge1'] = [score['rouge1'] for score in rouge_scores]
    data_new['rouge2'] = [score['rouge2'] for score in rouge_scores]
    data_new['rougeL'] = [score['rougeL'] for score in rouge_scores]
    data_new['rougeLSum'] = [score['rougeLSum'] for score in rouge_scores]

    # Tambahkan BERTScore per row
    data_new['bert_precision'] = [score['bert_precision'] for score in bert_scores]
    data_new['bert_recall'] = [score['bert_recall'] for score in bert_scores]
    data_new['bert_f1'] = [score['bert_f1'] for score in bert_scores]

    # Konversi ke DataFrame dan skalakan ke persen
    df_scores = pd.DataFrame(data_new)
    df_scores = df_scores * 100

    # Tambahkan kolom input, reference, dan generated
    df_scores['input'] = df_clean['input'].values
    df_scores['reference'] = df_clean['reference'].values
    df_scores['generated'] = df_clean['generated'].values

    # Hitung rata-rata skor
    mean_scores = df_scores.drop(columns=['input', 'reference', 'generated']).mean()

    return df_scores, mean_scores


In [ ]:
ft_model_6 = pd.read_csv('/content/Eval/model_ft_6/content/eval_basemodel_20250525_102353/fine_tuned_outputs.csv')
ft_model_6.sample(1)

,instruction,input,reference,generated,bert_precision,bert_recall,bert_f1
42,Konsumsi obat penambah darah ketika diare,Konsumsi obat penambah darah ketika diare. Bolehkah sangobion di komsumsi ketika sedang diare?,"pada umumnya, diare terjadi akibat konsumsi makanan atau minuman yang terkontaminasi bakteri, virus, atau parasit. dimana pada dewasa lebih sering karena bakteri, sedangkan pada anak anak lebih sering karena virus. selain karena infeksi, diare dapat muncul akibat : efek samping obat-obatan tertentu, faktor psikologi, misalnya gelisah, cemas, panik, dsb konsumsi minuman beralkohol dan kopi yang berlebihan. keadaan penyakit lain, seperti tifus, infeksi usus, peradangan usus buntu, dsb. konsums...","Kamu adalah asisten medis cerdas yang membantu menjawab pertanyaan kesehatan dengan informasi yang akurat dan terpercaya. Jika pertanyaannya di luar kemampuanmu, sarankan untuk konsultasi ke dokter. Jangan memberikan diagnosa langsung.<|eot_id|>\n\n### Keluhan:\nKonsumsi obat penambah darah ketika diare\n\n### Detail Pertanyaan:\nKonsumsi obat penambah darah ketika diare. Bolehkah sangobion di komsumsi ketika sedang diare?\n\n### Jawaban:\nDiare dapat disebabkan oleh banyak hal, seperti infe...",0.714864,0.714848,0.714856


In [ ]:
ft_model_6['generated'] = ft_model_6['generated'].apply(get_text_after_jawaban)

In [ ]:
ft_model_6.head()

,instruction,input,reference,generated,bert_precision,bert_recall,bert_f1
0,Solusi atasi sakit maag yang tak kunjung sembuh,"Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosanya sama yaitu saya menderita maag. Oleh dokter yang menangani, saya hanya diberi antasida, omeprazole, dan ranitidin. Saya sudah habiskan obatnya, pola makan pun saya jaga dengan tidak mengkonsumsi makanan pedas, asam, dan kafein namun sampai saat ini tidak ada perubahan. Selama 2 minggu saya sakit berat badan saya turun 1...","usus halus ataupun kerongkongan yang dapat disebabkan kondisi berikut : faktor psikis, yakni kondisi stres emosional akibat pengunaan obat antinyeri non steroid (nsaid) akibat perlukaan pada lambung (tukak lambung/peptik) infeksi bakteri helicobacter pylori (gastritis) dsb sakit maag yang kronis atau berlangsung berat dapat menyebabkan perlukaan pada lambung dan sebaliknya. pada perlukaan yang berat dapat terjadi perforasi (pecahnya) jaringan lambung sehingga menimbulkan gejala batuk/muntah ...","Kondisi yang anda alami dengan gejala utama berupa timbulnya rasa sakit pada perut bagian atas atau yang dalam bahasa medis disebut sebagai nyeri ulu hati, memang paling sering disebabkan oleh karena penyakit yang terkait organ lambung. Berikut ini beberapa penyakit yang dapat menyerang organ lambung, diantaranya adalah : gastritis=== peradangan pada dinding lambung bagian dalam, tukak peptik=== luka yang terjadi pada dinding lambung bagian dalam, penyakit asam lambung / gerd, kanker lambung...",0.681877,0.695313,0.688529
1,solusi untuk mengatasi konstipasi,"solusi untuk mengatasi konstipasi. dok kalo bab feses saya enggak keras, cair tapi usus saya enggak mau untuk ngeluarinnya. kenapa ya dok?",konstipasi adalah keadaan tidak dapat buang air besar secara teratur atau tidak dapat buang air besar sama sekali. konstipasi dapat disebabkan oleh: kurang mimun kurang konsumsi serat kurang aktif bergerak dan berolahraga kelebihan atau kekurangan berat badan kebiasaan menunda buang air besar cemas atau depresi penyumbatan usus gangguan saraf usus efek samping obat-obatan tertentu dsb. sebaiknya anda berkonsultasi dengan dokter di mana dokter akan menganalisis gejala dan melakukan pemeriksaa...,"Perlu anda ketahui sebelumnya bahwa konstipasi sendiri merupakan suatu kondisi yang ditandai dengan sulitnya buang air besar ( bab ) secara teratur, tidak bisa benar - benar tuntas, atau tidak bisa sama sekali. Berikut ini beberapa hal yang dapat menyebabkan konstipasi, yaitu : kurang minum, kurang konsumsi serat, perubahan pola makan, kebiasaan mengabaikan keinginan untuk bab, efek samping obat - obatan tertentu, gangguan mental seperti kecemasan atau depresi, dan lain sebagainya. Cobalah a...",0.648159,0.689129,0.668017
2,"Perut sakit, mual dan bersendawa berlebihan","Perut sakit, mual dan bersendawa berlebihan. saya ingin tany dok.. istri saya mrasakan sakit perut, kpala pusing di sertai mual2 dan bersendawa yang brlebih.. kira-kira apa pnyebab nya dan bgm solusi mngatasiny. terima kasih","penyebab sakit perut, mual dan bersendawa mungkin disebabkan oleh sin, yang gejala-gejalanya adalah: sering bersendawa tidak nyaman di perut, kembung mual penurunan nafsu makan dada terasa seperti terbakar penyebab dispepsia ini diantaranya: refluks asam lambung (gerd) ulkus peptikum intoleransi laktosa batu empedu dan radang empedu depresi dan kecemasan efek samping obat-obatan, misalnya: antibiotik, aspirin konsumsi kopi,dulu untuk memastikan diagnosis penyakit tersebut istri anda dapat be...","Perut yang terasa sakit, mual, pusing, dan bersendawa berlebihan bisa terjadi karena berbagai sebab. Beberapa kemungkinan yang bisa mencetuskan kondisi ini meliputi: gangguan pencernaan, contohnya dispepsia, radang usus, alergi atau malabsorpsi makanan, keracunan, gastroenteritis, sin usus, dan sebagainya gangguan pada sistem organ lain, contohnya gangguan 

In [ ]:
df_ft, mean_ft = evaluate_generation_scores(ft_model_6)

In [ ]:
df_ft.head(2)

,bleu1,bleu2,bleu3,bleu4,rouge1,rouge2,rougeL,rougeLSum,bert_precision,bert_recall,bert_f1,input,reference,generated
0,29.850377,15.115929,9.948291,7.026122,37.054632,9.069212,15.2019,15.2019,70.982909,66.013598,68.408126,"Solusi atasi sakit maag yang tak kunjung sembuh. Saya memiliki keluhan selama 2 minggu belakangan ada masalah di pencernaan. Saya sudah berobat klinik 2 kali tapi diagnosanya sama yaitu saya menderita maag. Oleh dokter yang menangani, saya hanya diberi antasida, omeprazole, dan ranitidin. Saya sudah habiskan obatnya, pola makan pun saya jaga dengan tidak mengkonsumsi makanan pedas, asam, dan kafein namun sampai saat ini tidak ada perubahan. Selama 2 minggu saya sakit berat badan saya turun 1...","usus halus ataupun kerongkongan yang dapat disebabkan kondisi berikut : faktor psikis, yakni kondisi stres emosional akibat pengunaan obat antinyeri non steroid (nsaid) akibat perlukaan pada lambung (tukak lambung/peptik) infeksi bakteri helicobacter pylori (gastritis) dsb sakit maag yang kronis atau berlangsung berat dapat menyebabkan perlukaan pada lambung dan sebaliknya. pada perlukaan yang berat dapat terjadi perforasi (pecahnya) jaringan lambung sehingga menimbulkan gejala batuk/muntah ...","Kondisi yang anda alami dengan gejala utama berupa timbulnya rasa sakit pada perut bagian atas atau yang dalam bahasa medis disebut sebagai nyeri ulu hati, memang paling sering disebabkan oleh karena penyakit yang terkait organ lambung. Berikut ini beberapa penyakit yang dapat menyerang organ lambung, diantaranya adalah : gastritis=== peradangan pada dinding lambung bagian dalam, tukak peptik=== luka yang terjadi pada dinding lambung bagian dalam, penyakit asam lambung / gerd, kanker lambung..."
1,33.891213,20.321421,13.133114,8.323199,43.023256,15.789474,25.0000,25.0000,70.742667,72.912562,71.811229,"solusi untuk mengatasi konstipasi. dok kalo bab feses saya enggak keras, cair tapi usus saya enggak mau untuk ngeluarinnya. kenapa ya dok?",konstipasi adalah keadaan tidak dapat buang air besar secara teratur atau tidak dapat buang air besar sama sekali. konstipasi dapat disebabkan oleh: kurang mimun kurang konsumsi serat kurang aktif bergerak dan berolahraga kelebihan atau kekurangan berat badan kebiasaan menunda buang air besar cemas atau depresi penyumbatan usus gangguan saraf usus efek samping obat-obatan tertentu dsb. sebaiknya anda berkonsultasi dengan dokter di mana dokter akan menganalisis gejala dan melakukan pemeriksaa...,"Perlu anda ketahui sebelumnya bahwa konstipasi sendiri merupakan suatu kondisi yang ditandai dengan sulitnya buang air besar ( bab ) secara teratur, tidak bisa benar - benar tuntas, atau tidak bisa sama sekali. Berikut ini beberapa hal yang dapat menyebabkan konstipasi, yaitu : kurang minum, kurang konsumsi serat, perubahan pola makan, kebiasaan mengabaikan keinginan untuk bab, efek samping obat - obatan tertentu, gangguan mental seperti kecemasan atau depresi, dan lain sebagainya. Cobalah a..."


In [ ]:
mean_ft

,0
bleu1,27.269446
bleu2,13.117255
bleu3,7.260492
bleu4,4.473363
rouge1,32.109918
rouge2,7.441203
rougeL,15.990500
rougeLSum,15.990500
bert_precision,69.130281
bert_recall,69.106900


In [ ]:
ft_model_6.to_csv('Hasil Eval revisi/eval_ft_model_6.csv')

In [ ]:
mean_ft.to_csv('Hasil Eval revisi/eval_ft_model_6_mean.csv')

In [ ]:
!zip -r /content/Hasil_Eval_revisi-6.zip /content/Hasil\ Eval\ revisi

  adding: content/Hasil Eval revisi/ (stored 0%)
  adding: content/Hasil Eval revisi/eval_basemodel_6_mean.csv (deflated 38%)
  adding: content/Hasil Eval revisi/eval_ft_model_6.csv (deflated 72%)
  adding: content/Hasil Eval revisi/eval_basemodel_6.csv (deflated 70%)
  adding: content/Hasil Eval revisi/eval_ft_model_6_mean.csv (deflated 40%)
